In [3]:
import boto3
import time
import csv
from pathlib import Path

textractor = boto3.client("textract", region_name="ap-southeast-1")

file_path = Path(r"/Users/isaaclim/Desktop/OCR/mentis-ocr/cth2.pdf")

if not file_path.exists():
    raise FileNotFoundError(f"The file {file_path} does not exist.")

with open(file_path, "rb") as document:
    imageBytes = bytearray(document.read())

response = textractor.analyze_document(
    Document={"Bytes": imageBytes}, FeatureTypes=["TABLES"]
)

print(response)

UnsupportedDocumentException: An error occurred (UnsupportedDocumentException) when calling the AnalyzeDocument operation: Request has unsupported document format

In [3]:
with open(file_path, "rb") as document:
    imageBytes = bytearray(document.read())

response = textractor.analyze_document(
    Document={"Bytes": imageBytes}, FeatureTypes=["TABLES"]
)

blocks = response["Blocks"]
tables = []

for block in blocks:
    if block["BlockType"] == "TABLE":
        table = []
        cell_map = {}
        for relationship in block["Relationships"]:
            if relationship["Type"] == "CHILD":
                for id in relationship["Ids"]:
                    cell_map[id] = None

        for cell in blocks:
            if cell["BlockType"] == "CELL" and cell["Id"] in cell_map:
                row_index = cell["RowIndex"]
                col_index = cell["ColumnIndex"]
                text = ""
                for relationship in cell["Relationships"]:
                    if relationship["Type"] == "CHILD":
                        for id in relationship["Ids"]:
                            for item in blocks:
                                if item["Id"] == id and item["BlockType"] == "WORD":
                                    text += item["Text"] + " "
                text = text.strip()
                cell_map[cell["Id"]] = (row_index, col_index, text)
                table.append((row_index, col_index, text))

        tables.append(table)

#csv_file_path = Path(r"D:\Engineer\Aws - sagemaker\sagemaker-mob\extracted_table.csv")
csv_file_path = Path(r"/Users/isaaclim/Desktop/OCR/mentis-ocr/cth2.pdf")


with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    for table in tables:
        max_row = max(cell[0] for cell in table)
        max_col = max(cell[1] for cell in table)
        table_data = [["" for _ in range(max_col)] for _ in range(max_row)]

        for cell in table:
            row_index, col_index, text = cell
            table_data[row_index - 1][col_index - 1] = text

        writer.writerows(table_data)
        writer.writerow([]) 

print(f"Output {csv_file_path}")

Extracted table data has been saved to D:\Engineer\Aws - sagemaker\sagemaker-mob\extracted_table.csv
